In [1]:
import torch
from torch import nn,optim

In [22]:
class logistic_regression(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(logistic_regression, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)
        self.thereshold = 0.5
        
    def forward(self, x):
        yhat = torch.sigmoid(self.linear(x))
        return yhat
    
    def predict(self, X):
        predictions = self.forward(X)
        return torch.where(predictions > self.thereshold, 1, 0)
    
    def accuracy(self, x_test, y_test):
        y_pred = self.predict(x_test)
        label = y_pred > 0.5
        return torch.mean((label == y_test.type(torch.ByteTensor)).type(torch.float))

In [7]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

### preparing the data

In [27]:
# Load Iris dataset
iris = load_iris()
_x = iris.data
_y = iris.target

# making the first flower class 0 and the second and third flower of class 1
_y[50:150] = 1
# stacking the new data together to use it for shuffling later
data = np.column_stack((_x, _y))
shufled_data = shuffle(data)

# new data after shuffling
x = shufled_data[:, 0:4]
y = shufled_data[:, 4]

# convert to tensors
X = torch.from_numpy(x.astype(np.float32))
Y = torch.from_numpy(y.astype(np.float32))

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

### setting the model

In [25]:
print(X_train.shape)
n_samples, n_features = X_train.shape

# setting up the model 
model = logistic_regression(n_features, 1)
optimizer = optim.SGD(model.parameters(), lr=0.1)
criterion = nn.BCELoss()

torch.Size([105, 4])


### training the model

In [30]:
num_epochs = 1000
def train_model(epochs, X, Y):    
    for epoch in range(epochs):
        yhat = model(X)
        loss = criterion(yhat, Y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if epoch % 100 == 0:
            print(f'Epoch [{epoch}/{epochs}], Loss: {loss.item():.4f}')
        
Y_train = Y_train.view(-1, 1)
train_model(num_epochs, X_train, Y_train)

# print(type(X_test))
with torch.no_grad():
    predictions = model.predict(X_test)
    # print("predictions: ", predictions)
    
    # Calculate accuracy 
    Y_test = Y_test.view(-1, 1)
    # print("Y_test: ", Y_test)
    acc = model.accuracy(X_test, Y_test)  
    print(f'accuracy: {acc.item():.4f}')

Epoch [0/1000], Loss: 0.0028
Epoch [100/1000], Loss: 0.0027
Epoch [200/1000], Loss: 0.0026
Epoch [300/1000], Loss: 0.0025
Epoch [400/1000], Loss: 0.0025
Epoch [500/1000], Loss: 0.0024
Epoch [600/1000], Loss: 0.0023
Epoch [700/1000], Loss: 0.0023
Epoch [800/1000], Loss: 0.0022
Epoch [900/1000], Loss: 0.0022
accuracy: 1.0000


In [33]:
print("SKLEARN MODEL")
model = LogisticRegression()
model.fit(X_train, Y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(Y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')

SKLEARN MODEL
Accuracy: 1.0000


C:\Users\menam\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
